##### Set kernel to Julia before running

This notebook handles simulation for data generation. Code is written in Julia because it can run ODE solvers faster than Python.

In [1]:
using DifferentialEquations, YAML
using Random, Distributions, DataFrames, CSV

In [2]:
function accel_const(velocity, acceleration, params)
    v_switch = params["v_switch"]
    v_min = params["v_min"]
    v_max = params["v_max"]
    a_max = params["a_max"]

    if velocity > v_switch
        pos_limit = a_max * v_switch / velocity

    else
        pos_limit = a_max
    end
    if(velocity <= v_min && acceleration <= 0) || (velocity >= v_max && acceleration >= 0)
        acceleration = 0.
    elseif acceleration <= -a_max
        acceleration = -a_max
    elseif acceleration >= pos_limit
        acceleration = pos_limit
    end
    return acceleration
end

accel_const (generic function with 1 method)

In [3]:
function steering_const(steering_angle, steering_velocity, params)

    s_min = params["s_min"]
    s_max = params["s_max"]
    sv_min = params["sv_min"]
    sv_max = params["sv_max"]

    if(steering_angle <= s_min && steering_velocity <= 0) || (steering_angle >= s_max && steering_velocity >= 0)
        steering_velocity = 0.
    elseif steering_velocity <= sv_min
        steering_velocity = sv_min
    elseif steering_velocity >= sv_max
        steering_velocity = sv_max
    end

    return steering_velocity
end

steering_const (generic function with 1 method)

In [4]:
function kinematic_model!(dx, x, u_initial, t)
    p = YAML.load_file("params.yml")
    Iz = p["I"]
    lr = p["lr"]
    lf = p["lf"]
    C_Sr = p["C_Sr"]
    C_Sf = p["C_Sf"]
    g = 9.81
    h = p["h"]
    m = p["m"]
    mu = p["mu"]
    u =[]

    append!(u, steering_const(x[3], u_initial[1], p))
    append!(u, accel_const(x[4], u_initial[2], p))

    dx[1] = x[4] * cos(x[5])
    dx[2] = x[4] * sin(x[5])
    dx[3] = u[1]
    dx[4] = u[2]
    dx[5] = x[4] / (lr + lf) * tan(x[3])

end
    

kinematic_model! (generic function with 1 method)

In [5]:
function simulate(model, num, filename)
    accel_dist = Uniform(-.5, .5)
    steer_vel_dist = Uniform(-.1, .1)
    start_time = 0
    end_time = 600
    dt = 0.01
    time = 0.
    update_every = 30.
    update_over = 15.
    
    Random.seed!(num)

    x = []
    x_names = []
    dx = []
    if model == kinematic_model!
        sx = 0.#rand(uniform)
        sy = 0.#rand(uniform)
        δ = 0.#rand(uniform)
        vel = 0.#rand(uniform)
        ψ = 0.#rand(uniform)
        x = [sx, sy, δ, vel, ψ]
        dx = [0., 0., 0., 0., 0.]
        x_names = ["time", "x", "y", "steering_angle", "velocity", "heading"]
    end
    u = [0., 0.]
    
    target_accel = 0.
    target_steer_vel = 0.
    accel_dif = 0.
    steer_vel_diff = 0.
    
    
    u_names = ["time", "steering_velocity", "acceleration"]
    x_wt = reshape(vcat(time, x), 1, length(x_names))
    u_wt = reshape(vcat(time, u), 1, length(u_names))

    deriv_wt = reshape(vcat(time, dx), 1, length(x_names))
    
    obs = DataFrame(x_wt, x_names)
    ins = DataFrame(u_wt, u_names)
    deriv = DataFrame(deriv_wt, x_names)

    while round(time, digits=3) < end_time
        
        if round(time, digits=3) % update_every == 0
            target_steer_vel = rand(steer_vel_dist)
            target_accel = rand(accel_dist)
            accel_dif = target_accel - u[2]
            steer_vel_diff = target_steer_vel - u[1]
        end
        
        
        # divide update into smaller steps
    
        if abs(u[2] - target_accel) > 0.001
            u[2] += (1 / (update_over / dt)) * accel_dif
        end
    
        if abs(u[1] - target_steer_vel) > 0.001
            u[1] += (1 / (update_over / dt)) * steer_vel_diff
        end
        
        
        ode = ODEProblem(model, x, (time, time+dt), u)

        sol = solve(ode, save_everystep = false)
        
        kinematic_model!(dx, x, u, time+dt)
        x = sol.u[2]

        time += dt
        x_wt = reshape(vcat(round(time, digits=3), x), 1, length(x)+1)
        deriv_wt = reshape(vcat(round(time, digits=3), dx), 1, length(dx)+1)
        u_wt = reshape(vcat(round(time, digits=3), u), 1, length(u)+1)
        push!(obs, x_wt)
        push!(ins, u_wt)
        push!(deriv, deriv_wt)
    
    end

    CSV.write("csv/obs_$(filename)_$(num).csv", obs)
    CSV.write("csv/in_$(filename)_$(num).csv", ins)
    CSV.write("csv/deriv_$(filename)_$(num).csv", deriv)
end

simulate (generic function with 1 method)

In [6]:
for i in 0:2
    simulate(kinematic_model!, i, "kinematic")
end
